# Generative Netzwerke

Rekurrente neuronale Netzwerke (RNNs) und ihre Varianten mit gated Zellen wie Long Short Term Memory Cells (LSTMs) und Gated Recurrent Units (GRUs) bieten einen Mechanismus für Sprachmodellierung, d.h. sie können die Reihenfolge von Wörtern lernen und Vorhersagen für das nächste Wort in einer Sequenz treffen. Dies ermöglicht es uns, RNNs für **generative Aufgaben** zu nutzen, wie z.B. gewöhnliche Textgenerierung, maschinelle Übersetzung und sogar Bildbeschriftung.

In der RNN-Architektur, die wir in der vorherigen Einheit besprochen haben, hat jede RNN-Einheit den nächsten versteckten Zustand als Ausgabe erzeugt. Wir können jedoch auch eine weitere Ausgabe zu jeder rekurrenten Einheit hinzufügen, die es uns ermöglicht, eine **Sequenz** auszugeben (die genauso lang ist wie die ursprüngliche Sequenz). Darüber hinaus können wir RNN-Einheiten verwenden, die bei jedem Schritt keine Eingabe akzeptieren, sondern nur einen anfänglichen Zustandsvektor nehmen und dann eine Sequenz von Ausgaben erzeugen.

In diesem Notebook konzentrieren wir uns auf einfache generative Modelle, die uns helfen, Text zu generieren. Der Einfachheit halber bauen wir ein **zeichenbasiertes Netzwerk**, das Text Buchstabe für Buchstabe generiert. Während des Trainings müssen wir einen Textkorpus nehmen und ihn in Buchstabenfolgen aufteilen.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

## Aufbau eines Zeichen-Vokabulars

Um ein generatives Netzwerk auf Zeichenebene zu erstellen, müssen wir den Text in einzelne Zeichen statt in Wörter aufteilen. Die `TextVectorization`-Schicht, die wir bisher verwendet haben, kann das nicht, daher haben wir zwei Möglichkeiten:

* Den Text manuell laden und die Tokenisierung "von Hand" durchführen, wie in [diesem offiziellen Keras-Beispiel](https://keras.io/examples/generative/lstm_character_level_text_generation/)
* Die `Tokenizer`-Klasse für die Tokenisierung auf Zeichenebene verwenden.

Wir entscheiden uns für die zweite Option. Mit `Tokenizer` kann man auch in Wörter tokenisieren, sodass man relativ einfach zwischen Tokenisierung auf Zeichen- und Wortebene wechseln kann.

Um eine Tokenisierung auf Zeichenebene durchzuführen, müssen wir den Parameter `char_level=True` übergeben:


In [2]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

tokenizer = keras.preprocessing.text.Tokenizer(char_level=True,lower=False)
tokenizer.fit_on_texts([x['title'].numpy().decode('utf-8') for x in ds_train])

Wir möchten auch ein spezielles Token verwenden, um das **Ende der Sequenz** zu kennzeichnen, das wir `<eos>` nennen werden. Lassen Sie uns dieses manuell zum Vokabular hinzufügen:


In [3]:
eos_token = len(tokenizer.word_index)+1
tokenizer.word_index['<eos>'] = eos_token

vocab_size = eos_token + 1

Um Text in Zahlenfolgen zu kodieren, können wir verwenden:


In [4]:
tokenizer.texts_to_sequences(['Hello, world!'])

[[48, 2, 10, 10, 5, 44, 1, 25, 5, 8, 10, 13, 78]]

## Training eines generativen RNN zur Erstellung von Titeln

Die Methode, mit der wir ein RNN trainieren, um Nachrichtentitel zu generieren, ist wie folgt: In jedem Schritt nehmen wir einen Titel, der in ein RNN eingespeist wird, und für jedes Eingabezeichen bitten wir das Netzwerk, das nächste Ausgabezeichen zu erzeugen:

![Bild zeigt ein Beispiel für die RNN-Generierung des Wortes 'HELLO'.](../../../../../lessons/5-NLP/17-GenerativeNetworks/images/rnn-generate.png)

Für das letzte Zeichen unserer Sequenz bitten wir das Netzwerk, das `<eos>`-Token zu generieren.

Der Hauptunterschied bei dem generativen RNN, das wir hier verwenden, besteht darin, dass wir die Ausgabe von jedem Schritt des RNN nehmen und nicht nur von der letzten Zelle. Dies kann erreicht werden, indem der Parameter `return_sequences` für die RNN-Zelle angegeben wird.

Während des Trainings wäre die Eingabe für das Netzwerk also eine Sequenz von codierten Zeichen einer bestimmten Länge, und die Ausgabe wäre eine Sequenz derselben Länge, jedoch um ein Element verschoben und mit `<eos>` abgeschlossen. Ein Minibatch besteht aus mehreren solchen Sequenzen, und wir müssen **Padding** verwenden, um alle Sequenzen auszurichten.

Lassen Sie uns Funktionen erstellen, die den Datensatz für uns transformieren. Da wir Sequenzen auf Minibatch-Ebene auffüllen möchten, werden wir den Datensatz zunächst durch Aufruf von `.batch()` gruppieren und ihn dann mit `map` transformieren. Die Transformationsfunktion nimmt also ein ganzes Minibatch als Parameter:


In [5]:
def title_batch(x):
    x = [t.numpy().decode('utf-8') for t in x]
    z = tokenizer.texts_to_sequences(x)
    z = tf.keras.preprocessing.sequence.pad_sequences(z)
    return tf.one_hot(z,vocab_size), tf.one_hot(tf.concat([z[:,1:],tf.constant(eos_token,shape=(len(z),1))],axis=1),vocab_size)

Einige wichtige Dinge, die wir hier tun:  
* Zuerst extrahieren wir den eigentlichen Text aus dem String-Tensor  
* `text_to_sequences` konvertiert die Liste von Strings in eine Liste von Integer-Tensoren  
* `pad_sequences` füllt diese Tensoren auf ihre maximale Länge auf  
* Schließlich führen wir eine One-Hot-Codierung aller Zeichen durch, verschieben sie und fügen `<eos>` hinzu. Wir werden bald sehen, warum wir One-Hot-codierte Zeichen benötigen  

Diese Funktion ist jedoch **Pythonic**, d.h. sie kann nicht automatisch in ein Tensorflow-Berechnungsdiagramm übersetzt werden. Wir erhalten Fehler, wenn wir versuchen, diese Funktion direkt in der `Dataset.map`-Funktion zu verwenden. Wir müssen diesen Pythonic-Aufruf mit dem `py_function`-Wrapper umschließen:  


In [6]:
def title_batch_fn(x):
    x = x['title']
    a,b = tf.py_function(title_batch,inp=[x],Tout=(tf.float32,tf.float32))
    return a,b

> **Hinweis**: Der Unterschied zwischen Pythonischen und Tensorflow-Transformationsfunktionen mag etwas zu komplex erscheinen, und Sie fragen sich vielleicht, warum wir den Datensatz nicht mit Standard-Python-Funktionen transformieren, bevor wir ihn an `fit` übergeben. Obwohl dies definitiv möglich ist, hat die Verwendung von `Dataset.map` einen großen Vorteil, da die Datenverarbeitungspipeline mit dem Tensorflow-Berechnungsgraphen ausgeführt wird. Dieser nutzt GPU-Berechnungen und minimiert die Notwendigkeit, Daten zwischen CPU und GPU hin- und herzuschieben.

Nun können wir unser Generator-Netzwerk erstellen und mit dem Training beginnen. Es kann auf jeder rekurrenten Zelle basieren, die wir in der vorherigen Einheit besprochen haben (einfach, LSTM oder GRU). In unserem Beispiel verwenden wir LSTM.

Da das Netzwerk Zeichen als Eingabe erhält und die Vokabulargröße relativ klein ist, benötigen wir keine Embedding-Schicht. Die One-Hot-kodierte Eingabe kann direkt in die LSTM-Zelle eingehen. Die Ausgabeschicht wäre ein `Dense`-Klassifikator, der die LSTM-Ausgabe in One-Hot-kodierte Token-Nummern umwandelt.

Außerdem, da wir es mit Sequenzen variabler Länge zu tun haben, können wir die `Masking`-Schicht verwenden, um eine Maske zu erstellen, die den gepolsterten Teil der Zeichenkette ignoriert. Dies ist nicht unbedingt erforderlich, da wir uns nicht besonders für alles interessieren, was über das `<eos>`-Token hinausgeht. Aber wir werden es verwenden, um etwas Erfahrung mit diesem Schichttyp zu sammeln. `input_shape` wäre `(None, vocab_size)`, wobei `None` die Sequenz variabler Länge angibt, und die Ausgabeschicht ist ebenfalls `(None, vocab_size)`, wie Sie aus der `summary` sehen können:


In [7]:
model = keras.models.Sequential([
    keras.layers.Masking(input_shape=(None,vocab_size)),
    keras.layers.LSTM(128,return_sequences=True),
    keras.layers.Dense(vocab_size,activation='softmax')
])

model.summary()
model.compile(loss='categorical_crossentropy')

model.fit(ds_train.batch(8).map(title_batch_fn))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, None, 84)          0         
_________________________________________________________________
lstm (LSTM)                  (None, None, 128)         109056    
_________________________________________________________________
dense (Dense)                (None, None, 84)          10836     
Total params: 119,892
Trainable params: 119,892
Non-trainable params: 0
_________________________________________________________________
15000/15000 [==============================] - 229s 15ms/step - loss: 1.5385


## Generieren von Ausgaben

Nachdem wir das Modell trainiert haben, möchten wir es verwenden, um einige Ausgaben zu erzeugen. Zunächst benötigen wir eine Möglichkeit, Text zu dekodieren, der durch eine Sequenz von Token-Nummern dargestellt wird. Dafür könnten wir die Funktion `tokenizer.sequences_to_texts` verwenden; allerdings funktioniert sie nicht gut mit einer Tokenisierung auf Zeichenebene. Daher nehmen wir ein Wörterbuch der Tokens aus dem Tokenizer (genannt `word_index`), erstellen eine umgekehrte Zuordnung und schreiben unsere eigene Dekodierungsfunktion:


In [10]:
reverse_map = {val:key for key, val in tokenizer.word_index.items()}

def decode(x):
    return ''.join([reverse_map[t] for t in x])

Nun beginnen wir mit einer Zeichenkette `start`, kodieren sie in eine Sequenz `inp`, und rufen dann bei jedem Schritt unser Netzwerk auf, um das nächste Zeichen zu bestimmen.

Die Ausgabe des Netzwerks `out` ist ein Vektor mit `vocab_size` Elementen, der die Wahrscheinlichkeiten jedes Tokens darstellt. Mit `argmax` können wir die Nummer des wahrscheinlichsten Tokens finden. Dieses Zeichen fügen wir dann der generierten Liste von Tokens hinzu und setzen die Generierung fort. Dieser Prozess, bei dem ein Zeichen generiert wird, wird `size`-mal wiederholt, um die benötigte Anzahl von Zeichen zu erzeugen. Die Generierung wird vorzeitig beendet, wenn das `eos_token` erreicht wird.


In [12]:
def generate(model,size=100,start='Today '):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            nc = tf.argmax(out)
            if nc==eos_token:
                break
            chars.append(nc.numpy())
            inp = inp+[nc]
        return decode(chars)
    
generate(model)

'Today #39;s lead to strike for the strike for the strike for the strike (AFP)'

## Ausgabe während des Trainings sampeln

Da wir keine nützlichen Metriken wie *Genauigkeit* haben, ist die einzige Möglichkeit, zu überprüfen, ob unser Modell besser wird, das **Sampeln** von generierten Zeichenketten während des Trainings. Dafür verwenden wir **Callbacks**, also Funktionen, die wir an die `fit`-Funktion übergeben können und die während des Trainings regelmäßig aufgerufen werden.


In [13]:
sampling_callback = keras.callbacks.LambdaCallback(
  on_epoch_end = lambda batch, logs: print(generate(model))
)

model.fit(ds_train.batch(8).map(title_batch_fn),callbacks=[sampling_callback],epochs=3)

Epoch 1/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.2703
Today #39;s a lead in the company for the strike
Epoch 2/3
15000/15000 [==============================] - 227s 15ms/step - loss: 1.2057
Today #39;s the Market Service on Security Start (AP)
Epoch 3/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.1752
Today #39;s a line on the strike to start for the start


Dieses Beispiel erzeugt bereits ziemlich guten Text, aber es gibt mehrere Möglichkeiten, ihn weiter zu verbessern:

* **Mehr Text**. Wir haben nur Titel für unsere Aufgabe verwendet, aber es könnte sinnvoll sein, mit vollständigem Text zu experimentieren. Beachten Sie, dass RNNs nicht besonders gut mit langen Sequenzen umgehen können. Daher macht es Sinn, entweder die Texte in kürzere Sätze aufzuteilen oder immer mit einer festen Sequenzlänge eines vordefinierten Werts `num_chars` (z. B. 256) zu trainieren. Sie könnten das obige Beispiel in eine solche Architektur umwandeln, indem Sie sich vom [offiziellen Keras-Tutorial](https://keras.io/examples/generative/lstm_character_level_text_generation/) inspirieren lassen.

* **Mehrschichtige LSTM**. Es könnte sinnvoll sein, 2 oder 3 Schichten von LSTM-Zellen auszuprobieren. Wie wir in der vorherigen Einheit erwähnt haben, extrahiert jede Schicht eines LSTM bestimmte Muster aus dem Text. Bei einem zeichenbasierten Generator können wir erwarten, dass die unteren LSTM-Schichten für die Extraktion von Silben verantwortlich sind, während die höheren Schichten Wörter und Wortkombinationen erkennen. Dies kann einfach implementiert werden, indem ein Parameter für die Anzahl der Schichten an den LSTM-Konstruktor übergeben wird.

* Sie könnten auch mit **GRU-Einheiten** experimentieren, um zu sehen, welche besser abschneiden, sowie mit **unterschiedlichen Größen der versteckten Schichten**. Eine zu große versteckte Schicht könnte zu Overfitting führen (z. B. lernt das Netzwerk den genauen Text auswendig), während eine kleinere Größe möglicherweise keine guten Ergebnisse liefert.


## Weiche Textgenerierung und Temperatur

In der vorherigen Definition von `generate` haben wir immer das Zeichen mit der höchsten Wahrscheinlichkeit als nächstes Zeichen im generierten Text ausgewählt. Dies führte oft dazu, dass sich der Text zwischen denselben Zeichenfolgen immer wieder "wiederholte", wie in diesem Beispiel:
```
today of the second the company and a second the company ...
```

Wenn wir uns jedoch die Wahrscheinlichkeitsverteilung für das nächste Zeichen ansehen, könnte es sein, dass der Unterschied zwischen den höchsten Wahrscheinlichkeiten nicht groß ist, z. B. kann ein Zeichen eine Wahrscheinlichkeit von 0,2 haben, ein anderes - 0,19 usw. Wenn wir beispielsweise das nächste Zeichen in der Sequenz '*play*' suchen, könnte das nächste Zeichen genauso gut ein Leerzeichen oder ein **e** sein (wie im Wort *player*).

Das führt uns zu der Erkenntnis, dass es nicht immer "fair" ist, das Zeichen mit der höchsten Wahrscheinlichkeit auszuwählen, da die Wahl des zweitwahrscheinlichsten Zeichens dennoch zu sinnvollem Text führen kann. Es ist klüger, Zeichen aus der Wahrscheinlichkeitsverteilung zu **samplen**, die durch die Netzwerkausgabe gegeben ist.

Dieses Sampling kann mit der Funktion `np.multinomial` durchgeführt werden, die die sogenannte **multinomiale Verteilung** implementiert. Eine Funktion, die diese **weiche** Textgenerierung umsetzt, ist unten definiert:


In [33]:
def generate_soft(model,size=100,start='Today ',temperature=1.0):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            probs = tf.exp(tf.math.log(out)/temperature).numpy().astype(np.float64)
            probs = probs/np.sum(probs)
            nc = np.argmax(np.random.multinomial(1,probs,1))
            if nc==eos_token:
                break
            chars.append(nc)
            inp = inp+[nc]
        return decode(chars)

words = ['Today ','On Sunday ','Moscow, ','President ','Little red riding hood ']
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"\n--- Temperature = {i}")
    for j in range(5):
        print(generate_soft(model,size=300,start=words[j],temperature=i))


--- Temperature = 0.3
Today #39;s strike #39; to start at the store return
On Sunday PO to Be Data Profit Up (Reuters)
Moscow, SP wins straight to the Microsoft #39;s control of the space start
President olding of the blast start for the strike to pay &lt;b&gt;...&lt;/b&gt;
Little red riding hood ficed to the spam countered in European &lt;b&gt;...&lt;/b&gt;

--- Temperature = 0.8
Today countie strikes ryder missile faces food market blut
On Sunday collores lose-toppy of sale of Bullment in &lt;b&gt;...&lt;/b&gt;
Moscow, IBM Diffeiting in Afghan Software Hotels (Reuters)
President Ol Luster for Profit Peaced Raised (AP)
Little red riding hood dace on depart talks #39; bank up

--- Temperature = 1.0
Today wits House buiting debate fixes #39; supervice stake again
On Sunday arling digital poaching In for level
Moscow, DS Up 7, Top Proble Protest Caprey Mamarian Strike
President teps help of roubler stepted lessabul-Dhalitics (AFP)
Little red riding hood signs on cash in Carter-youb

---

KeyError: 0

Wir haben einen weiteren Parameter namens **Temperatur** eingeführt, der angibt, wie strikt wir uns an die höchste Wahrscheinlichkeit halten sollten. Wenn die Temperatur 1,0 beträgt, führen wir eine faire multinomiale Stichprobe durch, und wenn die Temperatur gegen unendlich geht, werden alle Wahrscheinlichkeiten gleich, und wir wählen zufällig das nächste Zeichen aus. Im untenstehenden Beispiel können wir beobachten, dass der Text bedeutungslos wird, wenn wir die Temperatur zu stark erhöhen, und er ähnelt einem "zyklischen" hart generierten Text, wenn er sich näher an 0 bewegt.



---

**Haftungsausschluss**:  
Dieses Dokument wurde mit dem KI-Übersetzungsdienst [Co-op Translator](https://github.com/Azure/co-op-translator) übersetzt. Obwohl wir uns um Genauigkeit bemühen, beachten Sie bitte, dass automatisierte Übersetzungen Fehler oder Ungenauigkeiten enthalten können. Das Originaldokument in seiner ursprünglichen Sprache sollte als maßgebliche Quelle betrachtet werden. Für kritische Informationen wird eine professionelle menschliche Übersetzung empfohlen. Wir übernehmen keine Haftung für Missverständnisse oder Fehlinterpretationen, die sich aus der Nutzung dieser Übersetzung ergeben.
